In [1]:
import pandas as pd
import sqlite3
import nltk
from nltk.corpus import stopwords
import numpy as np

In [2]:
import sqlite3
con=sqlite3.connect("/Users/xiaoweichen/Kaggle/ClintonEmails/data/database.sqlite")

In [3]:
df=pd.read_sql_query("SELECT * FROM Emails",con)

In [ ]:
df.head(5)

In [5]:
# filter set contains all stop words
filter_set=set(stopwords.words("english"))
# feature set contains all single words (token) that are not stop words
feature_set=set()
# Go through all subjects
for subject in df.MetadataSubject.unique():
    tokens=nltk.word_tokenize(subject)
    tokens=[token for token in tokens if token not in filter_set]
    for token in tokens:
        feature_set.add(token)
    

In [6]:
df['MetadataSubject'][:3]

0                                                  WOW
1    H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...
2                                        CHRIS STEVENS
Name: MetadataSubject, dtype: object

In [7]:
emails=[]
persons=[]
from_=[]
to_=[]

In [8]:
for subject in df.MetadataSubject.unique():
    tokens=nltk.word_tokenize(subject)
    # Retrieve the sender info for the specific subject
    # and remove all semicolon from the sender info
    from_=[person.replace(";","") for person in df[df.MetadataSubject == subject].MetadataFrom.values if person != '']
    # Retrieve the receiver info for the specific subject
    # and remove all semicolon from the string of "To:"
    to_=[person.replace(";","") for person in df[df.MetadataSubject == subject].MetadataTo.values if person !='']
    # Note the use of set: remove duplicate elements
    from_.append(set(from_))
    to_.append(set(to_))
    # All contacts: sender and receiver
    persons.append(from_+to_)
    
    tokens=set([token for token in tokens if token not in filter_set])
    email=[1 if feature in tokens else 0 for feature in feature_set]
    emails.append(email)

In [9]:
from sklearn.cluster import MiniBatchKMeans

In [10]:
k=5
X=np.matrix(emails)
km=MiniBatchKMeans(n_clusters=k).fit(X)

In [ ]:
top_k_tags_per_cluster={}
for cluster,center in enumerate(km.cluster_centers_):
    